## Install Tensorflow

In [1]:
!pip install tensorflow

'load_ext' is not recognized as an internal or external command,
operable program or batch file.


## Import Libraries

In [2]:
import os
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf

## Prepping Data

In [3]:
df = pd.read_csv('detail_zip.csv',index_col=0)
temp_df = pd.read_csv('uszips.csv')
temp_df = temp_df[temp_df['state_id'] == 'IL'].reset_index(drop=True)
temp_df = temp_df[['zip','population','density']]
df = df.merge(temp_df, on='zip', how='left')
df['lead_level'] = (df['1st Draw'] + df['2/3 Min'] + df['5 Min']) / 3
df = df.drop(columns=['1st Draw', '2/3 Min', '5 Min'])
bins = [0, 1, 5, float('inf')]
labels = ['Low', 'Medium', 'High']
df['lead_level_cat'] = pd.cut(df['lead_level'], bins=bins, labels=labels)
df

,zip,median_INC_2017-2021,per_capita_INC_2017-2021,major_crime_2018-2022,violent_crime_2018-2022,public_crime_2012-2016,behavioral_ health_hospitalizations_2017,cognitive_difficulty_percent_2017-2021,poverty_rate_percent_2017-2021,high_school_grad_rate_2017-2021,college_grad_rate_2017-2021,non_hispanic_white_percent_2017-2021,population,density,lead_level,lead_level_cat
0,60601,123115.0,137556.0,1736.752366,148.889510,57.189159,91.0,1.995805,5.834093,97.917725,88.031149,54.877060,14804.0,14706.9,0.940404,Low
1,60602,235666.0,168423.0,282.033895,25.289746,5.774626,25.0,0.000000,2.222222,100.000000,82.217782,47.022767,1142.0,5350.7,1.464326,Medium
2,60605,112162.0,85947.0,1675.218975,246.799483,126.443697,285.0,2.914663,8.612424,95.316371,78.088010,57.037753,32077.0,8658.3,0.911358,Low
3,60606,130196.0,133605.0,529.774357,58.453847,20.751522,66.0,0.363857,8.550637,99.266177,91.794530,59.642207,3298.0,5487.1,1.533333,Medium
4,60607,109634.0,65052.0,2208.933790,266.924943,266.380039,651.0,4.155217,16.888971,96.620012,78.920985,51.797266,29990.0,5005.4,1.676312,Medium
5,60608,61490.0,26958.0,2208.318895,582.919388,978.662767,1137.0,3.625513,16.787975,74.265756,30.445646,20.245194,83689.0,5172.7,3.682467,Medium
6,60609,43048.0,23009.0,2179.855381,760.609450,1642.447833,1116.0,4.104459,26.924702,72.765042,17.377649,14.775060,61861.0,3082.1,5.531612,High
7,60610,99246.0,93795.0,1537.345214,220.706625,182.978152,498.0,3.149999,11.993181,97.158825,78.236189,69.706756,42422.0,14725.7,1.939278,Medium
8,60611,117344.0,125624.0,2285.077683,263.585238,86.933274,248.0,1.073433,8.912987,99.282669,87.964404,67.281986,34562.0,16829.4,1.000000,Low
9,60612,52126.0,37182.0,2377.324853,729.616162,1503.783816,1366.0,5.522375,29.057230,88.019560,39.508383,22.717353,33908.0,3520.5,4.150453,Medium


## Setting Up

In [13]:
X = df[['median_INC_2017-2021', 'per_capita_INC_2017-2021',
       'major_crime_2018-2022', 'violent_crime_2018-2022',
       'public_crime_2012-2016', 'behavioral_ health_hospitalizations_2017',
       'cognitive_difficulty_percent_2017-2021',
       'poverty_rate_percent_2017-2021', 'high_school_grad_rate_2017-2021',
       'college_grad_rate_2017-2021', 'non_hispanic_white_percent_2017-2021',
       'population', 'density']].astype(float)
y = df['lead_level_cat']

y = y.replace('Low', 0).replace('Medium', 1).replace('High', 2)

#print(y)

x_train = X.iloc[:42,:]
x_test = X.iloc[42:,:]

y_train = y.iloc[:42]
y_test = y.iloc[42:]

#print(y_train, y_test)

print(X.shape, y.shape)
print(len(X), len(y))

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
print(len(x_train), len(x_test))
print(len(y_train), len(y_test))

(53, 13) (53,)
53 53
(42, 13) (11, 13)
(42,) (11,)
42 11
42 11


In [14]:
def create_model1():
    return tf.keras.Sequential([tf.keras.layers.Dense(3, activation = 'linear'), 
                             tf.keras.layers.Dense(3, activation = 'linear'),
                             tf.keras.layers.Dense(5, activation = 'linear'),
                             tf.keras.layers.Dense(1, activation = 'linear')])

def create_model2():
    return tf.keras.Sequential([tf.keras.layers.Dense(3, activation = 'linear'), 
                             tf.keras.layers.Dense(5, activation = 'relu'),
                             tf.keras.layers.Dense(2, activation = 'linear'),
                             tf.keras.layers.Dense(1, activation = 'linear')])

def create_model3():
    return tf.keras.Sequential([tf.keras.layers.Dense(100, activation = 'relu'), 
                             tf.keras.layers.Dense(10, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')])

def train_models():
    

    model1 = create_model1()
    model1.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])
    
    model2 = create_model2()
    model2.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])
    
    model3 = create_model3()
    model3.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])

    logdir1 = os.path.join("logs1", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir2 = os.path.join("logs2", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir3 = os.path.join("logs3", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboardCallback1 = tf.keras.callbacks.TensorBoard(logdir1, histogram_freq=1)
    tensorboardCallback2 = tf.keras.callbacks.TensorBoard(logdir2, histogram_freq=1)
    tensorboardCallback3 = tf.keras.callbacks.TensorBoard(logdir3, histogram_freq=1)

    model1.fit(X, y, epochs = 100, callbacks = [tensorboardCallback1])
    model2.fit(X, y, epochs = 100, callbacks = [tensorboardCallback2])
    model3.fit(X, y, epochs = 100, callbacks = [tensorboardCallback3])
    
    
def train_models2():
    

    model1 = create_model1()
    model1.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])
    
    model2 = create_model2()
    model2.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])
    
    model3 = create_model3()
    model3.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
                optimizer = tf.keras.optimizers.SGD(), 
                metrics = ['accuracy'])

    logdir1 = os.path.join("logs4", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir2 = os.path.join("logs5", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir3 = os.path.join("logs6", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboardCallback1 = tf.keras.callbacks.TensorBoard(logdir1, histogram_freq=1)
    tensorboardCallback2 = tf.keras.callbacks.TensorBoard(logdir2, histogram_freq=1)
    tensorboardCallback3 = tf.keras.callbacks.TensorBoard(logdir3, histogram_freq=1)

    model1.fit(X, y, epochs = 100, validation_data = (x_test, y_test), callbacks = [tensorboardCallback1])
    model2.fit(X, y, epochs = 100, validation_data = (x_test, y_test), callbacks = [tensorboardCallback2])
    model3.fit(X, y, epochs = 100, validation_data = (x_test, y_test), callbacks = [tensorboardCallback3])


In [15]:
tf.random.set_seed(1)
train_models2()


Epoch 1/100



C:\Users\nic12\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)



1/2 [==============>...............] - ETA: 0s - loss: 1.6019e-07 - accuracy: 0.4688

C:\Users\nic12\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 1s 327ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 2/100
2/2 [==============================] - 0s 87ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 3/100
2/2 [==============================] - 0s 96ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 4/100
2/2 [==============================] - 0s 85ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 5/100
2/2 [==============================] - 0s 84ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 6/100
2/2 [==============================] - 0s 96ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-07 - val_accuracy: 0.6364
Epoch 7/100
2/2 [==============================] - 0s 87ms/step - loss: 1.5295e-07 - accuracy: 0.5660 - val_loss: 1.4088e-0

...

C:\Users\nic12\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 1s 238ms/step - loss: 0.0000e+00 - accuracy: 0.3585 - val_loss: 0.0000e+00 - val_accuracy: 0.6364
Epoch 2/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5312

C:\Users\nic12\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 0s 84ms/step - loss: nan - accuracy: 0.5660 - val_loss: nan - val_accuracy: 0.0909
Epoch 3/100
2/2 [==============================] - 0s 81ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 4/100
2/2 [==============================] - 0s 82ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 5/100
2/2 [==============================] - 0s 91ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 6/100
2/2 [==============================] - 0s 90ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 7/100
2/2 [==============================] - 0s 85ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 8/100
2/2 [==============================] - 0s 81ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 9/100
2/2 [==============================] - 0s 83ms/step - loss: na

Epoch 63/100
2/2 [==============================] - 0s 92ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 64/100
2/2 [==============================] - 0s 88ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 65/100
2/2 [==============================] - 0s 84ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 66/100
2/2 [==============================] - 0s 89ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 67/100
2/2 [==============================] - 0s 90ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 68/100
2/2 [==============================] - 0s 89ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 69/100
2/2 [==============================] - 0s 89ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0909
Epoch 70/100
2/2 [==============================] - 0s 

In [21]:
%load_ext tensorboard
# %tensorboard --logdir logs2
%tensorboard --logdir logsAllDenamed

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 11736), started 0:03:58 ago. (Use '!kill 11736' to kill it.)